In [1]:
import pandas as pd

In [2]:
data = pd.read_pickle('input_data.pickle')

print(data)

          result  odds_home  odds_draw  odds_away points_home points_away  \
index                                                                       
50.0    home win       1.20       6.00      15.00          15           3   
51.0    home win       1.13       8.00      19.00          15           6   
52.0        draw       2.10       3.30       3.50           6           4   
53.0    home win       1.67       3.75       5.00           5           3   
54.0    away win      10.00       5.50       1.29           3          13   
...          ...        ...        ...        ...         ...         ...   
3415.0      draw       1.44       4.33       7.50           8           2   
3416.0  away win       3.30       3.40       2.20           3           7   
3417.0  away win       2.10       4.00       3.00          10           7   
3418.0  home win       2.50       3.30       2.87           7          10   
3419.0  away win       2.62       3.30       2.70           7           8   